In [1]:
import os
import joblib
import numpy as np
import pandas as pd
import polars as pl

import pandas.api.types
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

import lightgbm as lgb
import xgboost as xgb
import catboost as cb

import lifelines
from lifelines.utils import concordance_index

import sys

print(sys.version)
for i in [np, pd, pl, mpl, sns, lifelines, sklearn, lgb, xgb, cb]:
    try:
        print(i.__name__, i.__version__)
    except:
        print(i.__name__)

3.12.6 (main, Sep 30 2024, 02:19:13) [GCC 9.4.0]
numpy 1.26.4
pandas 2.2.3
polars 1.12.0
matplotlib 3.8.4
seaborn 0.13.2
lifelines 0.30.0
sklearn 1.5.2
lightgbm
xgboost 2.1.2
catboost 1.2.5


In [18]:
import dproc, sgutil, sgpp, sgml, custpp
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate, KFold, ShuffleSplit, train_test_split
from sklearn.impute import SimpleImputer
from lifelines import NelsonAalenFitter

data_path = 'data'
model_path = 'model'

def score(df, prds):
    return df.groupby('race_group', observed=True).apply(
        lambda x: concordance_index(x['efs_time'], -prds.loc[x.index], x['efs']), include_groups=False
    ).pipe(
        lambda x: float(x.mean() - x.std(ddof=0))
    )
p2 =joblib.load(os.path.join(model_path, 'p2.joblib'))
df_train = p2.transform([os.path.join(data_path, 'train.csv')])

In [8]:
df_kmf_cv = pd.concat([
    joblib.load(os.path.join(model_path, '{}.cv'.format(i)))['cv_best_']['prd'].sort_index().rename(i) for i in ['lgb1', 'xgb1', 'cb1']
], axis=1)
df_kmf_cv.head()

,lgb1,xgb1,cb1
ID,,,
0,0.483554,0.497868,0.492672
1,0.620134,0.632588,0.685708
2,0.434267,0.439699,0.449668
3,0.660707,0.650732,0.661139
4,0.644397,0.648778,0.664089


In [9]:
df_naf_cv = pd.concat([
    joblib.load(os.path.join(model_path, '{}.cv'.format(i)))['cv_best_']['prd'].sort_index().rename(i) for i in ['lgb2', 'xgb2', 'cb2']
], axis=1)
df_naf_cv.head()

,lgb2,xgb2,cb2
ID,,,
0,-0.725706,-0.712117,-0.723394
1,-0.497212,-0.503058,-0.445457
2,-0.802942,-0.803226,-0.785873
3,-0.444087,-0.466340,-0.468297
4,-0.488937,-0.472672,-0.462788


In [43]:
df_stk = pd.concat([
    df_kmf_cv.dot([0.4, 0.3, 0.3]).rename('kmf').rank(),
    df_naf_cv.dot([0.4, 0.3, 0.3]).rename('naf').rank()
], axis=1)

In [44]:
score(df_train, df_stk.dot([0.0, 1.0]))

0.6772404254003302